In [1]:
%load_ext autoreload
%autoreload 2

# Ar and Xe in vacuum

In [2]:
import molsysmt as msm
import numpy as np
from tqdm import tqdm
import openmm as mm
import openmm.app as app
import openmm.unit as unit

In [3]:
# System definition.

## First atom: Argon
mass_1 = 39.948 * unit.amu
sigma_1 = 3.404 * unit.angstroms
epsilon_1 = 0.238 * unit.kilocalories_per_mole
charge_1 = 0.0 * unit.elementary_charge

## Second atomo: Xenon
mass_2 = 131.293 * unit.amu
sigma_2 = 3.961 * unit.angstroms
epsilon_2 = 0.459 * unit.kilocalories_per_mole
charge_2 = 0.0 * unit.elementary_charge


# System creation.

system = mm.System()

non_bonded_force = mm.NonbondedForce()

reduced_sigma = 0.5*(sigma_1+sigma_2)
cutoff_distance = 4.0*reduced_sigma
switching_distance = 3.0*reduced_sigma
non_bonded_force.setNonbondedMethod(mm.NonbondedForce.CutoffPeriodic)
non_bonded_force.setUseSwitchingFunction(True)
non_bonded_force.setCutoffDistance(cutoff_distance)
non_bonded_force.setSwitchingDistance(switching_distance)

# Adding atom 1
system.addParticle(mass_1)
non_bonded_force.addParticle(charge_1, sigma_1, epsilon_1)

# Adding atom 2
system.addParticle(mass_2)
non_bonded_force.addParticle(charge_2, sigma_2, epsilon_2)

# Periodic box

system.setDefaultPeriodicBoxVectors([3.0, 0.0, 0.0]*unit.nanometers, [0.0, 3.0, 0.0]*unit.nanometers, [0.0, 0.0, 3.0]*unit.nanometers)

_ = system.addForce(non_bonded_force)

In [4]:
# Thermodynamic state and integrator.

step_size = 2*unit.femtoseconds
temperature = 300*unit.kelvin
friction = 1.0/unit.picosecond

integrator = mm.LangevinIntegrator(temperature, friction, step_size)

In [5]:
# Platform.

platform_name = 'CUDA'
platform = mm.Platform.getPlatformByName(platform_name)

In [6]:
# Contexto.

context = mm.Context(system, integrator, platform)

In [7]:
# Initial conditions.

initial_positions  = np.zeros([2, 3], np.float32) * unit.angstroms
initial_velocities = np.zeros([2, 3], np.float32) * unit.angstroms/unit.picoseconds

initial_positions[1, 0] = 1.0 * unit.nanometers

context.setPositions(initial_positions)
context.setVelocities(initial_velocities)

In [8]:
# Simulation parameters.

simulation_time = 20.0*unit.nanosecond
saving_time = 1.0*unit.picoseconds

n_steps_per_saving_period = int(saving_time/step_size)
n_saving_periods = int(simulation_time/saving_time)

In [9]:
# Numpy arrays as simulation reporters.

time = np.zeros([n_saving_periods], np.float32) * unit.picoseconds
position = np.zeros([n_saving_periods, 2, 3], np.float32) * unit.nanometers
velocity = np.zeros([n_saving_periods, 2, 3], np.float32) * unit.nanometers/unit.picosecond
potential_energy   = np.zeros([n_saving_periods], np.float32) * unit.kilocalories_per_mole
kinetic_energy     = np.zeros([n_saving_periods], np.float32) * unit.kilocalories_per_mole
box = np.zeros([n_saving_periods, 3, 3], np.float32) * unit.nanometers

In [10]:
# Saving data for time 0 in reporters

state = context.getState(getPositions=True, getVelocities=True, getEnergy=True)
time[0] = state.getTime()
position[0] = state.getPositions()
velocity[0] = state.getVelocities()
kinetic_energy[0]=state.getKineticEnergy()
potential_energy[0]=state.getPotentialEnergy()
box[0] = state.getPeriodicBoxVectors()

In [11]:
# Running the simulation

for ii in tqdm(range(n_saving_periods)):
    context.getIntegrator().step(n_steps_per_saving_period)
    state = context.getState(getPositions=True, getVelocities=True, getEnergy=True)
    time[ii] = state.getTime()
    position[ii] = state.getPositions()
    velocity[ii] = state.getVelocities()
    kinetic_energy[ii]=state.getKineticEnergy()
    potential_energy[ii]=state.getPotentialEnergy()
    box[ii] = state.getPeriodicBoxVectors()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [04:55<00:00, 67.77it/s]


## molsysmt.TrajectoryDict class and trjdpk file

In [12]:
trajdict={
    'time' : time,
    'coordinates' : position,
    'box' : box
          }

In [13]:
msm.get_form(trajdict)

'molsysmt.TrajectoryDict'

In [14]:
msm.convert(trajdict, to_form='Ar_Xe_pbc_vacuum.trjpk')

'Ar_Xe_traj.trjpk'

In [15]:
trajdict2 = msm.convert('Ar_Xe_pbc_vacuum.trjpk', to_form='molsysmt.TrajectoryDict')

In [16]:
trajdict2

{'coordinates': array([[[ 2.30269432e-02, -2.16523647e-01,  2.03206614e-02],
         [ 1.03093898e+00,  5.56341708e-02, -4.41431999e-03]],
 
        [[ 1.87892094e-01, -4.66710091e-01,  2.48895824e-01],
         [ 1.09362710e+00,  1.06191061e-01,  7.02466443e-02]],
 
        [[ 2.84138054e-01, -4.00966406e-01,  3.16997290e-01],
         [ 1.19666910e+00,  1.35289863e-01, -7.95007963e-03]],
 
        ...,
 
        [[-7.06794662e+01, -5.36857643e+01, -1.36262665e+01],
         [ 3.24669504e+00,  1.16425133e+01,  1.26712828e+01]],
 
        [[-7.08386917e+01, -5.38223267e+01, -1.35910883e+01],
         [ 3.07422066e+00,  1.16331511e+01,  1.25415907e+01]],
 
        [[-7.06279068e+01, -5.40050278e+01, -1.34346027e+01],
         [ 2.93348694e+00,  1.15726280e+01,  1.26575241e+01]]],
       dtype=float32) <Unit('nanometer')>,
 'box': array([[[3., 0., 0.],
         [0., 3., 0.],
         [0., 0., 3.]],
 
        [[3., 0., 0.],
         [0., 3., 0.],
         [0., 0., 3.]],
 
        [[3., 0